# Library

In [ ]:
!pip install transformers
!pip install Sastrawi
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=95d94db1d906ab90a056e1111db8dd7376b60875daa757bf7821d26d273cee2d
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
# import library untuk memuat dan menganalisis data
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

# import library untuk text cleaning
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re

# import library untuk deteksi bahasa
from langdetect import detect, LangDetectException

# import pipeline
from sklearn.pipeline import Pipeline

# import library untuk membuat model
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline
from tqdm import tqdm
import torch
import os

# import library untuk menyimpan model yang sudah dibuat
import joblib

# Memuat Data

In [ ]:
Data = "https://github.com/keaganwjy/datathon/raw/refs/heads/main/reviews_genshin_impact_raw.csv"
jumlah_baris = 2000 # jumlah baris yang akan dimasukan.as_integer_ratio
df_raw = pd.read_csv(Data) # membaca CSV
df = df_raw.head(jumlah_baris).copy() # menggunakan .head() untuk mengambil baris teratas secara langsung
df.head()

,Nama Pengguna,Teks Ulasan,Rating Bintang,Tanggal Ulasan,Jumlah Likes
0,Pengguna Google,pelit pengen karakter aja susah malah dapet yg...,1,2025-06-20 14:40:33,0
1,Pengguna Google,farming artefak engga pernah dikasih stat/sub ...,1,2025-06-20 14:00:33,0
2,Pengguna Google,"udah ngga worth it, perbaiki diri mu genshin, ...",1,2025-06-20 13:49:19,0
3,Pengguna Google,kasih fitur skip.,1,2025-06-20 13:22:21,0
4,Pengguna Google,peak update,5,2025-06-20 13:16:04,0


# Data Cleaning Dengan Pipeline

1. Penyesuaian Data
2. Pembersihan Text
3. Deteksi Bahasa

In [ ]:
# class untuk menyesuaikan tipe data
class PenyesuaianTipeData(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # mengubah 'Tanggal Ulasan' menjadi format datetime
        # ini memungkinkan kita untuk melakukan analisis berbasis waktu nanti
        X['Tanggal Ulasan'] = pd.to_datetime(X['Tanggal Ulasan'])

        # memastikan 'Rating Bintang' dan 'Jumlah Likes' adalah angka (integer)
        # error='coerce' akan mengubah nilai yang tidak bisa diubah menjadi angka (misal: teks) menjadi NaN (kosong)
        X['Rating Bintang'] = pd.to_numeric(X['Rating Bintang'], errors='coerce')
        X['Jumlah Likes'] = pd.to_numeric(X['Jumlah Likes'], errors='coerce')

        # kita bisa membuang baris yang rating atau likes-nya menjadi NaN, atau mengisinya dengan 0
        X.dropna(subset=['Rating Bintang'], inplace=True) # rating wajib ada
        # kode baru (praktik terbaik dan aman)
        X['Jumlah Likes'] = X['Jumlah Likes'].fillna(0)

        # mengubah tipe data menjadi integer untuk efisiensi memori
        X['Rating Bintang'] = X['Rating Bintang'].astype(int)
        X['Jumlah Likes'] = X['Jumlah Likes'].astype(int)

        return X

In [ ]:
# class untuk pembersihan teks
class PembersihanText(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # inisialisasi stemmer dan stopword remover
        factory_stemmer = StemmerFactory()
        stemmer = factory_stemmer.create_stemmer()
        factory_stopword = StopWordRemoverFactory()
        stopword_remover = factory_stopword.create_stop_word_remover()

        # buat kamus slang yang lebih lengkap (ini bisa terus Anda kembangkan)
        slang_dict = {
            # Original Anda yang sudah bagus
            'bgt': 'banget', 'gak': 'tidak', 'ga': 'tidak', 'gk':'tidak', 'tak':'tidak', 'kalo': 'kalau', 'gaca': 'gacha',
            'dgn': 'dengan', 'krn': 'karena', 'yg': 'yang', 'utk': 'untuk', 'mantep': 'mantap', 'mantapp': 'mantap',
            'keren': 'keren', 'bug': 'bug', 'ngebug': 'bug', 'loding': 'loading', 'ngelek': 'lag',
            'ngeframe': 'frame', 'ngedrop': 'drop', 'jlk': 'jelek', 'bangettt': 'banget',
            'sih': '', 'kok': '', 'ni':'ini', 'anying':'anjing', # 'nya' saya hapus untuk lebih aman, biarkan stemmer yang urus
            'udh':'sudah', 'udah':'sudah', 'dah':'sudah', 'dh':'sudah','ku':'aku', 'dikit':'sedikit', 'dpt':'dapat', 'dapet':'dapat',
            'thn':'tahun', 'ksh':'kasih', 'dr':'dari', 'pake':'pakai', 'blm':'belum',
            'tetep':'tetap', 'laen':'lain', 'mls':'malas','males':'malas', 'sampe':'sampai',
            'cmn':'cuman', 'lgi':'lagi', 'gitu':'begitu', 'gw':'gua', 'pls':'please',

            # Koreksi
            'gjls':'tidak jelas',

            # Penambahan Baru (Saran)
            # Istilah Game & RPG
            'char': 'karakter', 'stat': 'status', 'dmg': 'damage', 'exp': 'experience', 'lvl': 'level',
            'eq': 'equipment', 'equip': 'equipment', 'p2w': 'pay to win', 'f2p': 'free to play', 'rng': 'random',

            # Variasi Kata Kerja
            'ngeheal': 'heal', 'ngetank': 'tank', 'ngecheat': 'cheat', 'grinding': 'grinding', 'farming': 'farming',

            # Typo & Singkatan Umum
            'bgs': 'bagus', 'bngt': 'banget', 'cpt': 'cepat', 'trus': 'terus', 'bnyk': 'banyak',
            'grafiknya':'grafik', 'grafikny':'grafik', 'gamenya':'game', 'gameny':'game',
            'ceritanya':'cerita', 'ceritany':'cerita', 'jg': 'juga', 'jga': 'juga', 'sm': 'sama'
        }

        def preprocess_text(text):
            # pastikan input adalah string
            if not isinstance(text, str):
                return ""
            # 1. case folding
            text = text.lower()
            # 2. hapus noise (URL, mention, hashtag, karakter non-alfabet)
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\@\w+|\#', '', text)
            text = re.sub(r'[^a-z\s]', '', text)
            # 3. normalisasi kata slang
            words = text.split()
            normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
            text = " ".join(normalized_words)
            # 4. Stopword Removal
            text = stopword_remover.remove(text)
            # 5. stemming
            text = stemmer.stem(text)
            return text

        # menggunakan .copy() untuk menghindari SettingWithCopyWarning
        X_processed = X.copy()
        X_processed['Ulasan Bersih'] = X_processed['Teks Ulasan'].apply(preprocess_text)
        return X_processed


In [ ]:
class DeteksiBahasa(BaseEstimator, TransformerMixin):
    """
    Class transformer untuk mendeteksi bahasa pada kolom teks
    dan memfilter DataFrame untuk hanya menyimpan baris dengan bahasa target.
    """
    def __init__(self, input_column='Ulasan Bersih', target_language='id'):
        """
        Inisialisasi dengan kolom input dan bahasa target yang diinginkan.
        """
        print("Inisialisasi DeteksiBahasa...")
        self.input_column = input_column
        self.target_language = target_language
        print(f"Akan memfilter untuk bahasa: '{self.target_language}'")

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X harus berupa Pandas DataFrame.")

        X_transformed = X.copy()

        print(f"Memulai deteksi bahasa pada kolom '{self.input_column}'...")
        tqdm.pandas(desc="Deteksi Bahasa")
        X_transformed['bahasa'] = X_transformed[self.input_column].progress_apply(self._detect_language_safe)

        print("\nDistribusi bahasa sebelum filtering:")
        print(X_transformed['bahasa'].value_counts())

        # Filter DataFrame
        original_rows = len(X_transformed)
        result_df = X_transformed[X_transformed['bahasa'] == self.target_language].copy()
        filtered_rows = original_rows - len(result_df)

        print(f"\nFiltering selesai. {filtered_rows} baris yang bukan '{self.target_language}' telah dibuang.")

        # --- PERBAIKAN DITAMBAHKAN DI SINI ---
        # Reset indeks agar kembali berurutan dari 0
        result_df.reset_index(drop=True, inplace=True)
        print("Indeks telah di-reset.")

        return result_df

    def _detect_language_safe(self, text):
        """Fungsi helper yang aman untuk mendeteksi satu teks."""
        if not isinstance(text, str) or len(text.strip()) < 15: # Tingkatkan threshold untuk akurasi
            return 'pendek/kosong'

        try:
            return detect(text)
        except LangDetectException:
            return 'error'

# Labelling

In [ ]:
class Labelling(BaseEstimator, TransformerMixin):
    """
    Class transformer untuk melakukan pelabelan otomatis multi-label.
    Dirancang untuk digunakan dalam Scikit-learn Pipeline.
    Input: DataFrame.
    Output: DataFrame yang sama dengan penambahan kolom-kolom label biner.
    """
    def __init__(self, input_column='Ulasan Bersih', model_name="facebook/bart-large-mnli", threshold=0.6):
        self.input_column = input_column
        self.model_name = model_name
        self.threshold = threshold
        self.candidate_labels = ['Cerita', 'Gameplay', 'Grafis', 'Bugs & Error', 'Optimalisasi', 'Monetisasi & Gacha', 'Komunitas']

        # Inisialisasi komponen yang 'mahal' hanya sekali
        print(f"Menginisialisasi Labelling: Memuat model {self.model_name}...")
        self.classifier = pipeline("zero-shot-classification",
                                   model=self.model_name,
                                   device=0 if torch.cuda.is_available() else -1)

        self.mlb = MultiLabelBinarizer(classes=self.candidate_labels)
        print("Inisialisasi Labelling selesai.")

    def fit(self, X, y=None):
        # Fit tidak melakukan apa-apa, hanya mengembalikan objek itu sendiri
        return self

    def transform(self, X, y=None):
        """
        Menerima Pandas Series berisi teks, mengembalikan DataFrame dengan label biner.
        """
        if not isinstance(X, pd.Series):
            raise TypeError("Input X harus berupa Pandas Series (satu kolom DataFrame).")

        print(f"Memulai pelabelan otomatis pada {len(X)} data...")

        # 1. Dapatkan prediksi list label untuk setiap teks
        tqdm.pandas(desc="Pelabelan Otomatis")
        list_of_labels = X.progress_apply(self._get_labels_for_text)

        print("Pelabelan otomatis selesai. Memulai binarisasi...")

        # 2. Lakukan binarisasi pada hasil list label
        binary_labels = self.mlb.fit_transform(list_of_labels)

        # --- PERBAIKAN UTAMA DI SINI ---

        # 3. Buat DataFrame baru dari hasil biner DENGAN NAMA KOLOM ASLI
        df_labels = pd.DataFrame(binary_labels, columns=self.mlb.classes_)

        # 4. SETELAH DATAFRAME DIBUAT, baru tambahkan prefix ke kolom-kolomnya
        df_labels = df_labels.add_prefix('AutoLabel_')

        # --------------------------------

        print("Binarisasi dan penambahan prefix selesai.")
        return df_labels

    def _get_labels_for_text(self, text):
        """Fungsi helper untuk memprediksi satu teks."""
        if not isinstance(text, str) or not text.strip():
            return []
        try:
            result = self.classifier(text, self.candidate_labels, multi_label=True)
            return [label for label, score in zip(result['labels'], result['scores']) if score > self.threshold]
        except Exception:
            return []

# Model

In [ ]:
class Modelling(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Gunakan GPU jika tersedia
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load IndoBERT
        tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
        model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1").to(device)
        model.eval()

        # buat label sentimen manual (bisa juga diganti dengan anotasi jika tersedia)
        def simple_sentiment(text):
            text = text.lower()
            if any(k in text for k in ['jelek', 'buruk', 'crash', 'lag', 'sampah']):
                return "negatif"
            elif any(k in text for k in ['bagus', 'keren', 'mantap', 'seru']):
                return "positif"
            else:
                return "netral"

        # terapkan label sentimen
        X['sentimen'] = X['Ulasan Bersih'].apply(simple_sentiment)

        # encode label sentimen
        label_encoder = LabelEncoder()
        X['label_sentimen'] = label_encoder.fit_transform(X['sentimen']) # 0 = negatif, 1 = netral, 2 = positif

        # melakukan feature extractor untuk IndoBERT
        def get_bert_embedding(text, max_len=128):
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=max_len)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
            # Ambil CLS token sebagai representasi kalimat
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
            return cls_embedding

        # ambil embedding untuk seluruh data
        print("Mengekstrak embedding IndoBERT...")
        embeddings = np.array([get_bert_embedding(text) for text in tqdm(X['Ulasan Bersih'])])

        # latih model machine learning
        X_mod = embeddings
        y_mod = label_encoder.fit_transform(X['sentimen'])

        X_train_mod, X_test_mod, y_train_mod, y_test_mod = train_test_split(X_mod, y_mod, test_size=0.2, random_state = 42)

        # latih model dengan Logistic Regression
        clf = LogisticRegression(max_iter=1000)
        clf.fit(X_train_mod, y_train_mod)

        y_pred = clf.predict(X_test_mod)

        # Evaluasi
        print("\nHasil evaluasi model sentimen")
        print(classification_report(y_test_mod, y_pred, target_names=label_encoder.classes_))

        # prediksi dan kategorisasi game
        # terapkan prediksi ke seluruh data
        X['pred_sentimen'] = label_encoder.inverse_transform(clf.predict(X_mod))

        # Buat ringkasan per kategori berdasarkan label zero-shot sebelumnya
        from collections import Counter

        def ringkasan_kategori(X):
            hasil = []
            candidate = ['Cerita', 'Gameplay', 'Grafis', 'Bugs & Error',
                        'Optimalisasi', 'Monetisasi & Gacha', 'Komunitas']
            for kategori in candidate:
                subset = X[X['auto_labels_list'].apply(lambda x: kategori in x)]
                total = len(subset)
                if total == 0:
                    continue
                counter = Counter(subset['pred_sentimen'])
                ringkas = {
                    'Kategori': kategori,
                    'Total Ulasan': total,
                    'Positif': counter['positif'],
                    'Negatif': counter['negatif'],
                    'Netral': counter['netral'],
                }
                hasil.append(ringkas)
            return pd.DataFrame(hasil)

        df_summary = ringkasan_kategori(X)
        print("\nRingkasan Sentimen per Kategori:")
        print(df_summary)

        # prediksi tingkat akurasi model
        accuracy = accuracy_score(y_test_mod,y_pred)
        print(f"\nTingkat akurasi model: {accuracy}")

        return X

# Execute

In [ ]:
Pipe = Pipeline([
    ("Penyesuaian",PenyesuaianTipeData()),
    ("Pembersihan",PembersihanText()),
    ("Deteksi",DeteksiBahasa()),
])

Inisialisasi DeteksiBahasa...
Akan memfilter untuk bahasa: 'id'


In [ ]:
model = Pipe.fit_transform(df)

Memulai deteksi bahasa pada kolom 'Ulasan Bersih'...


Deteksi Bahasa: 100%|██████████| 2000/2000 [00:06<00:00, 294.07it/s]


Distribusi bahasa sebelum filtering:
bahasa
id               1117
pendek/kosong     517
tl                121
en                 97
lt                 16
sl                 15
no                 14
sw                 11
et                  9
ca                  9
nl                  8
tr                  8
so                  8
da                  7
af                  7
sq                  6
hr                  5
fi                  3
ro                  3
pl                  3
it                  3
de                  2
fr                  2
es                  2
hu                  2
lv                  1
cy                  1
sv                  1
pt                  1
sk                  1
Name: count, dtype: int64

Filtering selesai. 883 baris yang bukan 'id' telah dibuang.
Indeks telah di-reset.


In [ ]:
model.to_csv('DataYangSudahClean.csv')

In [ ]:
model = pd.read_csv("DataYangSudahClean.csv") # membaca CSV
DataLabel = Labelling(input_column='Ulasan Bersih')
model2 = DataLabel.fit_transform(model['Ulasan Bersih'])

Menginisialisasi Labelling: Memuat model facebook/bart-large-mnli...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Inisialisasi Labelling selesai.
Memulai pelabelan otomatis pada 1117 data...


Pelabelan Otomatis: 100%|██████████| 1117/1117 [03:09<00:00,  5.89it/s]

Pelabelan otomatis selesai. Memulai binarisasi...
Binarisasi dan penambahan prefix selesai.


In [ ]:
model2.to_csv('DataYangSudahAdaLabel.csv', index=False)